# 🏥 Healthcare Analytics & Predictive Modeling

Offline project: EDA + models (LogReg, RF, GB), saves best model for the Streamlit app.

In [ ]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.inspection import permutation_importance
import joblib

df = pd.read_csv('../data/patients.csv')
df.head()

In [ ]:
df.describe().T

In [ ]:
ax = df['age'].hist(bins=20, figsize=(5,3)); plt.title('Age Distribution'); plt.tight_layout(); plt.savefig('../outputs/age_hist.png'); plt.show()

In [ ]:
X = df.drop(columns=['target']); y = df['target']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=7, stratify=y)
num_cols = X.columns.tolist()
pre = ColumnTransformer([('scale', StandardScaler(), num_cols)], remainder='passthrough')

models = {
    'logreg': LogisticRegression(max_iter=500),
    'rf': RandomForestClassifier(n_estimators=300, random_state=7),
    'gb': GradientBoostingClassifier(random_state=7)
}

scores={}
for k, clf in models.items():
    pipe = Pipeline([('pre', pre), ('clf', clf)])
    pipe.fit(X_train, y_train)
    proba = pipe.predict_proba(X_test)[:,1]
    scores[k]=roc_auc_score(y_test, proba)
scores

In [ ]:
best_key = max(scores, key=scores.get)
print('Best model:', best_key, 'AUC=', round(scores[best_key],3))
from sklearn.metrics import roc_curve
pipe = Pipeline([('pre', pre), ('clf', models[best_key])])
pipe.fit(X_train, y_train)
import joblib
joblib.dump(pipe, '../models/best_model.joblib')
fpr, tpr, _ = roc_curve(y_test, pipe.predict_proba(X_test)[:,1])
plt.figure(figsize=(6,5)); plt.plot(fpr,tpr,label=f"{best_key} AUC={scores[best_key]:.3f}")
plt.plot([0,1],[0,1],'--'); plt.legend(); plt.tight_layout(); plt.savefig('../outputs/roc_curve.png'); plt.show()

In [ ]:
perm = permutation_importance(pipe, X_test, y_test, n_repeats=10, random_state=7)
imp = pd.DataFrame({'feature': X.columns, 'importance': perm.importances_mean}).sort_values('importance', ascending=False)
imp.head(10)